## A gentle 10-minute primer to Ray AI Runitime (Ray AIR)

© 2019-2022, Anyscale. All Rights Reserved

📖 [Back to Table of Contents](./ex_00_tutorial_overview.ipynb)<br>
⬅️ [Previous notebook](./ex_07_ray_data.ipynb) <br>

### Overview

As part of Ray 2.0, Ray AI Runtime (AIR) is an open-source and unified toolkit for building end-to-end simple and scalable ML applications. 

Ray AI Runtime focuses on two functional aspects:
 * It provides scalability by leveraging Ray’s distributed compute layer for ML workloads.
 * It is designed to interoperate with other systems for storage and metadata needs.

Ray AIR consists of five key components:

 * Data processing ([Ray Data](https://docs.ray.io/en/latest/data/dataset.html))
 * Model Training ([Ray Train](https://docs.ray.io/en/latest/train/train.html))
 * Hyperparameter Tuning ([Ray Tune](https://docs.ray.io/en/latest/tune/index.html))
 * Model Serving ([Ray Serve](https://docs.ray.io/en/latest/serve/index.html)).
 * Reinforcement Learning ([Ray RLlib](https://docs.ray.io/en/latest/rllib/index.html))
 
 <img src = "images/ray-air.svg" width="60%" height="30%">
 
📖 [Back to Table of Contents](./ex_00_tutorial_overview.ipynb)<br>
⬅️ [Previous notebook](./ex_07_ray_data.ipynb) <br>
 
### Learning objectives:
  * How to use Ray AIR as a unified toolkit to write an end-to-end ML application in a single Python script
  * Use out-of-box Preprocessors
  * Load model from the best model checkpoint and use for batch inference
  * Deploy best checkpoint model and use for online inference

In [1]:
import logging, os, random, warnings
import ray
import pandas as pd

In [2]:
warnings.filterwarnings("ignore")
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
if ray.is_initialized:
    ray.shutdown()
ray.init(logging_level=logging.ERROR)

Python version:,3.8.13
Ray version:,2.0.0rc1
Dashboard:,http://127.0.0.1:8265


### End-to-end ML stages for a Ray AIR ML application

<img src="images/ray_air_pipeline.png" width="50%" height="25%">

### 1. Create Ray data from an S3 CSV datasource

In [4]:
dataset = ray.data.read_csv("s3://anonymous@air-example-data/breast_cancer.csv")

# Split data into train and validation.
train_dataset, valid_dataset = dataset.train_test_split(test_size=0.3)
test_dataset = valid_dataset.drop_columns(["target"])

Map_Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 17.61it/s]


### 2. Use out-of-the-box Preprocessors
This preprocessor is automatically used in the training function to `fit` and `tranform` your datasets for training and validation. You don't have to explicitly call the preprocess before training or inference. Ray AIR toolkit automatically does that for you. 

We are going to scaler a few features like `mean radius` and `mean texture`.

In [5]:
from ray.data.preprocessors import StandardScaler

# Create a preprocessor to scale some columns
columns_to_scale = ["mean radius", "mean texture"]
preprocessor = StandardScaler(columns=columns_to_scale)

### 3a. Use AIR Trainers for supported ML frameworks
Use the Ray AIR trainer `XGBoostTrainer` with simple steps:
 1. define the parallelism for Ray compute
 2. define the XGBoost parameters for training
 3. supply the preprocessor for fitting and transforming dataset during training and validation
 4. provide the datasets for training and validation
 5. invoke `trainer.fit()` 
 
 Simple API that does a lot behind the scenes for you!

In [6]:
from ray.air.config import ScalingConfig
from ray.train.xgboost import XGBoostTrainer

trainer = XGBoostTrainer(
    scaling_config=ScalingConfig(
        # Number of workers to use for data parallelism.
        num_workers=2,
        # Whether to use GPU acceleration.
        use_gpu=False),
    label_column="target",
    num_boost_round=20,
    params={
        # XGBoost specific params
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
    },
    # our train and validation dataset and preprocessor
    datasets={"train": train_dataset, "valid": valid_dataset},
    preprocessor=preprocessor,
)

##### Fit the trainer

In [7]:
result = trainer.fit()
# print(result.metrics)

Trial name,status,loc,iter,total time (s),train-logloss,train-error,valid-logloss
XGBoostTrainer_36724_00000,TERMINATED,127.0.0.1:58162,21,4.73242,0.0184957,0,0.0893879


(XGBoostTrainer pid=58162) 2022-08-14 18:18:49,999	INFO main.py:980 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
(_RemoteRayXGBoostActor pid=58177) 2022-08-14 18:18:51,188	WARNING __init__.py:192 -- DeprecationWarning: `ray.worker.get_resource_ids` is a private attribute and access will be removed in a future Ray version.
(_RemoteRayXGBoostActor pid=58177)   File "/opt/miniconda3/envs/ray-summit-training/lib/python3.8/site-packages/ray/_private/workers/default_worker.py", line 237, in <module>
(_RemoteRayXGBoostActor pid=58177)     ray._private.worker.global_worker.main_loop()
(_RemoteRayXGBoostActor pid=58177)   File "/opt/miniconda3/envs/ray-summit-training/lib/python3.8/site-packages/ray/_private/worker.py", line 754, in main_loop
(_RemoteRayXGBoostActor pid=58177)     self.core_worker.run_task_loop()
(_RemoteRayXGBoostActor pid=58177)   File "/opt/miniconda3/envs/ray-summit-training/lib/python3.8/site-packages/ray/_private/func

Result for XGBoostTrainer_36724_00000:
  date: 2022-08-14_18-18-53
  done: false
  experiment_id: 83d4aa5f8fc9479abf482a6cd9528f30
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 58162
  time_since_restore: 4.143232107162476
  time_this_iter_s: 4.143232107162476
  time_total_s: 4.143232107162476
  timestamp: 1660526333
  timesteps_since_restore: 0
  train-error: 0.02261306532663317
  train-logloss: 0.464117960489575
  training_iteration: 1
  trial_id: '36724_00000'
  valid-error: 0.11695906432748537
  valid-logloss: 0.5025240946234318
  warmup_time: 0.002746105194091797
  


(XGBoostTrainer pid=58162) 2022-08-14 18:18:53,349	INFO main.py:1516 -- [RayXGBoost] Finished XGBoost training on training data with total N=398 in 3.36 seconds (1.82 pure XGBoost training time).


Result for XGBoostTrainer_36724_00000:
  date: 2022-08-14_18-18-53
  done: true
  experiment_id: 83d4aa5f8fc9479abf482a6cd9528f30
  experiment_tag: '0'
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 21
  node_ip: 127.0.0.1
  pid: 58162
  time_since_restore: 4.732424974441528
  time_this_iter_s: 0.45326900482177734
  time_total_s: 4.732424974441528
  timestamp: 1660526333
  timesteps_since_restore: 0
  train-error: 0.0
  train-logloss: 0.01849572773292735
  training_iteration: 21
  trial_id: '36724_00000'
  valid-error: 0.04093567251461988
  valid-logloss: 0.08938791319913073
  warmup_time: 0.002746105194091797
  


### 3b. Use AIR Tuner for hyperparameter search

What if you want to do hyperparameter optimization during training and use the best config for the model? Well, you can then use Tuner and supply your training function, Trainer, as part of the argument, along 
with other Tuner configuration. 

Again, simple steps:
 1. define your hyperparameter space
 2. define `TuneConfig` for number of trials and parallelism 
 3. invoke `tuner.fit()`

In [8]:
from ray import tune

param_space = {"params": {"max_depth": tune.randint(1, 9)}}
metric = "train-logloss"
our_mode="min"

In [9]:
from ray.tune.tuner import Tuner, TuneConfig
from ray.air.config import RunConfig

tuner = Tuner(
    trainer,
    param_space=param_space,
    tune_config=TuneConfig(num_samples=5, metric=metric, mode=our_mode),
)
# Execute tuning.
result_grid = tuner.fit()

Trial name,status,loc,params/max_depth,iter,total time (s),train-logloss,train-error,valid-logloss
XGBoostTrainer_3c51c_00000,TERMINATED,127.0.0.1:58193,2,21,2.94402,0.0405455,0.00502513,0.0916641
XGBoostTrainer_3c51c_00001,TERMINATED,127.0.0.1:58208,2,21,5.03086,0.0405455,0.00502513,0.0916641
XGBoostTrainer_3c51c_00002,TERMINATED,127.0.0.1:58209,3,21,5.07783,0.0215151,0,0.0765915
XGBoostTrainer_3c51c_00003,TERMINATED,127.0.0.1:58313,7,21,4.79629,0.0184957,0,0.0893879
XGBoostTrainer_3c51c_00004,TERMINATED,127.0.0.1:58327,2,21,6.02368,0.0405455,0.00502513,0.0916641


(XGBoostTrainer pid=58193) 2022-08-14 18:18:58,801	INFO main.py:980 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
(_RemoteRayXGBoostActor pid=58214) 2022-08-14 18:19:00,071	WARNING __init__.py:192 -- DeprecationWarning: `ray.worker.get_resource_ids` is a private attribute and access will be removed in a future Ray version.
(_RemoteRayXGBoostActor pid=58214)   File "/opt/miniconda3/envs/ray-summit-training/lib/python3.8/site-packages/ray/_private/workers/default_worker.py", line 237, in <module>
(_RemoteRayXGBoostActor pid=58214)     ray._private.worker.global_worker.main_loop()
(_RemoteRayXGBoostActor pid=58214)   File "/opt/miniconda3/envs/ray-summit-training/lib/python3.8/site-packages/ray/_private/worker.py", line 754, in main_loop
(_RemoteRayXGBoostActor pid=58214)     self.core_worker.run_task_loop()
(_RemoteRayXGBoostActor pid=58214)   File "/opt/miniconda3/envs/ray-summit-training/lib/python3.8/site-packages/ray/_private/func

Result for XGBoostTrainer_3c51c_00000:
  date: 2022-08-14_18-19-01
  done: false
  experiment_id: 7d071c3a6df449a58c97e8599d22956c
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 58193
  time_since_restore: 2.566265106201172
  time_this_iter_s: 2.566265106201172
  time_total_s: 2.566265106201172
  timestamp: 1660526341
  timesteps_since_restore: 0
  train-error: 0.04773869346733668
  train-logloss: 0.4862994935794092
  training_iteration: 1
  trial_id: 3c51c_00000
  valid-error: 0.09941520467836257
  valid-logloss: 0.5120853461020174
  warmup_time: 0.002655029296875
  


(XGBoostTrainer pid=58193) 2022-08-14 18:19:01,372	INFO main.py:1516 -- [RayXGBoost] Finished XGBoost training on training data with total N=398 in 2.59 seconds (1.11 pure XGBoost training time).
(_RemoteRayXGBoostActor pid=58239) 2022-08-14 18:19:01,476	WARNING __init__.py:192 -- DeprecationWarning: `ray.worker.get_resource_ids` is a private attribute and access will be removed in a future Ray version.
(_RemoteRayXGBoostActor pid=58239)   File "/opt/miniconda3/envs/ray-summit-training/lib/python3.8/site-packages/ray/_private/workers/default_worker.py", line 237, in <module>
(_RemoteRayXGBoostActor pid=58239)     ray._private.worker.global_worker.main_loop()
(_RemoteRayXGBoostActor pid=58239)   File "/opt/miniconda3/envs/ray-summit-training/lib/python3.8/site-packages/ray/_private/worker.py", line 754, in main_loop
(_RemoteRayXGBoostActor pid=58239)     self.core_worker.run_task_loop()
(_RemoteRayXGBoostActor pid=58239)   File "/opt/miniconda3/envs/ray-summit-training/lib/python3.8/sit

Result for XGBoostTrainer_3c51c_00000:
  date: 2022-08-14_18-19-01
  done: true
  experiment_id: 7d071c3a6df449a58c97e8599d22956c
  experiment_tag: 0_max_depth=2
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 21
  node_ip: 127.0.0.1
  pid: 58193
  time_since_restore: 2.944024085998535
  time_this_iter_s: 0.2989687919616699
  time_total_s: 2.944024085998535
  timestamp: 1660526341
  timesteps_since_restore: 0
  train-error: 0.0050251256281407
  train-logloss: 0.04054545671047278
  training_iteration: 21
  trial_id: 3c51c_00000
  valid-error: 0.02923976608187134
  valid-logloss: 0.09166410522894901
  warmup_time: 0.002655029296875
  


...

Result for XGBoostTrainer_3c51c_00003:
  date: 2022-08-14_18-19-09
  done: true
  experiment_id: 3c46af3e8ed841719ba45842f072e6a2
  experiment_tag: 3_max_depth=7
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 21
  node_ip: 127.0.0.1
  pid: 58313
  time_since_restore: 4.79629373550415
  time_this_iter_s: 0.5882058143615723
  time_total_s: 4.79629373550415
  timestamp: 1660526349
  timesteps_since_restore: 0
  train-error: 0.0
  train-logloss: 0.01849572773292735
  training_iteration: 21
  trial_id: 3c51c_00003
  valid-error: 0.04093567251461988
  valid-logloss: 0.08938791319913073
  warmup_time: 0.0026509761810302734
  


(XGBoostTrainer pid=58327) 2022-08-14 18:19:10,561	INFO main.py:1025 -- [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=58361) [18:19:10] task [xgboost.ray]:4841700848 got new rank 0
(_RemoteRayXGBoostActor pid=58362) [18:19:10] task [xgboost.ray]:4886445744 got new rank 1


Result for XGBoostTrainer_3c51c_00004:
  date: 2022-08-14_18-19-12
  done: false
  experiment_id: 5ee52894336e49899713194b896738b4
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 58327
  time_since_restore: 5.170567274093628
  time_this_iter_s: 5.170567274093628
  time_total_s: 5.170567274093628
  timestamp: 1660526352
  timesteps_since_restore: 0
  train-error: 0.04773869346733668
  train-logloss: 0.4862994935794092
  training_iteration: 1
  trial_id: 3c51c_00004
  valid-error: 0.09941520467836257
  valid-logloss: 0.5120853461020174
  warmup_time: 0.002862691879272461
  


(XGBoostTrainer pid=58327) 2022-08-14 18:19:12,332	INFO main.py:1516 -- [RayXGBoost] Finished XGBoost training on training data with total N=398 in 4.11 seconds (1.77 pure XGBoost training time).


Result for XGBoostTrainer_3c51c_00004:
  date: 2022-08-14_18-19-13
  done: true
  experiment_id: 5ee52894336e49899713194b896738b4
  experiment_tag: 4_max_depth=2
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 21
  node_ip: 127.0.0.1
  pid: 58327
  time_since_restore: 6.02367901802063
  time_this_iter_s: 0.7711498737335205
  time_total_s: 6.02367901802063
  timestamp: 1660526353
  timesteps_since_restore: 0
  train-error: 0.0050251256281407
  train-logloss: 0.04054545671047278
  training_iteration: 21
  trial_id: 3c51c_00004
  valid-error: 0.02923976608187134
  valid-logloss: 0.09166410522894901
  warmup_time: 0.002862691879272461
  


In [10]:
# Fetch the best result with its best hyperparameter config 
best_result = result_grid.get_best_result()
print("Best Result:", best_result)

Best Result: Result(metrics={'train-logloss': 0.01849572773292735, 'train-error': 0.0, 'valid-logloss': 0.08938791319913073, 'valid-error': 0.04093567251461988, 'done': True, 'trial_id': '3c51c_00003', 'experiment_tag': '3_max_depth=7'}, error=None, log_dir=PosixPath('/Users/jules/ray_results/XGBoostTrainer_2022-08-14_18-18-57/XGBoostTrainer_3c51c_00003_3_max_depth=7_2022-08-14_18-19-03'))


### Ray AIR Checkpoints

The AIR trainers, tuners, and custom pretrained model generate Checkpoints. An AIR Checkpoint is a format for models that are used across different components of the Ray AI Runtime. This common format allows easy interoperability among AIR components and seamless integration with external supported machine learning frameworks. Read more
about [Checkpoints]().

<img src="images/checkpoints.jpeg" height="25%" and width="50%"> 

### 4. Use AIR `BatchPreditor` for batch prediction
Once you have trained and tuned your model, create a batch predictor from best model using the `best_result.checkpoint` and do batch inference. 

In [11]:
from ray.train.batch_predictor import BatchPredictor
from ray.train.xgboost import XGBoostPredictor

batch_predictor = BatchPredictor.from_checkpoint(best_result.checkpoint, XGBoostPredictor)

predicted_probabilities = batch_predictor.predict(test_dataset)
print("PREDICTED PROBABILITIES")
predicted_probabilities.show()

Map Progress (1 actors 0 pending): 100%|███████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.31it/s]

PREDICTED PROBABILITIES
{'predictions': 0.9964648485183716}
{'predictions': 0.9951295852661133}
{'predictions': 0.0037899704184383154}
{'predictions': 0.9964648485183716}
{'predictions': 0.9969868063926697}
{'predictions': 0.9947494864463806}
{'predictions': 0.9899886250495911}
{'predictions': 0.9952162504196167}
{'predictions': 0.3375702202320099}
{'predictions': 0.9766711592674255}
{'predictions': 0.0037899704184383154}
{'predictions': 0.9948934316635132}
{'predictions': 0.9472665786743164}
{'predictions': 0.989780068397522}
{'predictions': 0.9952002763748169}
{'predictions': 0.18953870236873627}
{'predictions': 0.2149435132741928}
{'predictions': 0.99428790807724}
{'predictions': 0.9890844225883484}
{'predictions': 0.0037899704184383154}


### 5. Use `PredictorDeployment` for online inference

Deploy the best model as an inference service by using Ray Serve and the `PredictorDeployment` class.

In [12]:
from ray import serve
from fastapi import Request
from ray.serve import PredictorDeployment
from ray.serve.http_adapters import pandas_read_json

serve.run(
    PredictorDeployment.options(name="XGBoostService", num_replicas=2, route_prefix="/rayair").bind(
        XGBoostPredictor, result.checkpoint, http_adapter=pandas_read_json
    )
)

(ServeController pid=58605) INFO 2022-08-14 18:21:45,720 controller 58605 http_state.py:129 - Starting HTTP proxy with name 'SERVE_CONTROLLER_ACTOR:SERVE_PROXY_ACTOR-52f3a76f3beccffde9ca71005cd8c247efc586679d8768a349b60bc4' on node '52f3a76f3beccffde9ca71005cd8c247efc586679d8768a349b60bc4' listening on '127.0.0.1:8000'
(HTTPProxyActor pid=58607) INFO:     Started server process [58607]
(ServeController pid=58605) INFO 2022-08-14 18:21:46,337 controller 58605 deployment_state.py:1232 - Adding 2 replicas to deployment 'XGBoostService'.


RayServeSyncHandle(deployment='XGBoostService')

After deploying the service, you can send requests to it.

In [16]:
import requests

sample_input = test_dataset.take(1)
sample_input = dict(sample_input[0])

output = requests.post("http://localhost:8000/rayair", json=[sample_input]).json()
print(output)

[{'predictions': 0.9964648485183716}]


(HTTPProxyActor pid=58607) INFO 2022-08-14 18:24:41,568 http_proxy 127.0.0.1 http_proxy.py:315 - POST /rayair 307 2.3ms
(HTTPProxyActor pid=58607) INFO 2022-08-14 18:24:41,581 http_proxy 127.0.0.1 http_proxy.py:315 - POST /rayair 200 11.1ms
(ServeReplica:XGBoostService pid=58609) INFO 2022-08-14 18:24:41,580 XGBoostService XGBoostService#RVTcck replica.py:482 - HANDLE __call__ OK 9.2ms
(ServeReplica:XGBoostService pid=58610) INFO 2022-08-14 18:24:41,567 XGBoostService XGBoostService#kxXvxt replica.py:482 - HANDLE __call__ OK 0.2ms


In [18]:
ray.shutdown()

### Homework

1. Have a go at Ray AIR examples in the documentation.

 📖 [Back to Table of Contents](./ex_00_tutorial_overview.ipynb)<br>
⬅️ [Previous notebook](./ex_07_ray_data.ipynb) <br>

Done! 🍻
 